In [4]:
option = 'H100_80GB_HBM3'
!mkdir -p notes/capture/{option}
!mkdir -p notes/data/{option}
!meson test -C build/ hamiltonian-batch --verbose -t 0 > notes/capture/{option}/batchgpu.txt

In [7]:
# Enable all CPU cores
from tqdm import tqdm
import pandas as pd
import re
import pandas as pd

# Read the file content
def parse_to_datafame(file_path):
  with open(file_path, "r") as file:
      content = file.read()
  # Define a regex pattern to extract the test case and its details
  pattern = r"\s+cpu_time\s+(?P<cpu_time>[\d.]+)\s+gpu_gb_in\s+(?P<gpu_gb_in>[\d.]+)\s+gpu_gb_out\s+(?P<gpu_gb_out>[\d.]+)\s+gpu_gb_total\s+(?P<gpu_gb_total>[\d.]+)\s+gpu_between_atoms\s+(?P<gpu_between_atoms>[\d.]+)\s+gpu_in_atoms\s+(?P<gpu_in_atoms>[\d.]+)\s+gpu_time\s+(?P<gpu_time>[\d.]+)\s+gpu_transfer_time\s+(?P<gpu_transfer_time>[\d.]+)\s+gpu_walltime\s+(?P<gpu_walltime>[\d.]+)\s+batch_size\s+(?P<batch_size>[\d.]+)"

  # Use re.finditer to extract all matches
  matches = re.finditer(pattern, content)

  # Create a list of dictionaries to store the extracted data
  data = []
  for match in matches:
      data.append(match.groupdict())

  # Convert the list of dictionaries into a pandas DataFrame
  df = pd.DataFrame(data)

# Convert numeric columns to appropriate data types
  numeric_columns = [
    'cpu_time', 'gpu_gb_in', 'gpu_gb_out', 'gpu_gb_total',
    'gpu_between_atoms', 'gpu_in_atoms', 'gpu_time', 'gpu_transfer_time', "batch_size"
  ]
  # # Convert the numeric columns to float
  df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric)

  # Display the DataFrame
  return df

df = parse_to_datafame(f"notes/capture/{option}/batchgpu.txt")
df.to_csv(f"notes/data/{option}/batchgpu.csv", index=False)
df

,cpu_time,gpu_gb_in,gpu_gb_out,gpu_gb_total,gpu_between_atoms,gpu_in_atoms,gpu_time,gpu_transfer_time,gpu_walltime,batch_size
0,23.504586,0.00106,0.001046,0.002105,5.476960,0.385376,5.862336,1.426080,1072.3413107916713,1
1,29.719504,0.00106,0.001046,0.002105,0.317312,0.689280,1.323904,2.037120,5.4114935919642448,2
2,47.255725,0.00106,0.001046,0.002105,0.129616,0.248672,0.767136,1.502592,2.8033191338181496,4
3,21.731307,0.00106,0.001046,0.002105,0.085680,0.254144,0.939584,1.485504,2.9148990288376808,8
4,76.633493,0.00106,0.001046,0.002105,0.069880,0.259744,1.377824,1.561120,3.4534763544797897,16
5,78.350078,0.00106,0.001046,0.002105,0.063966,0.320800,2.367712,1.943936,4.7418596222996712,32
6,20.886853,0.00106,0.001046,0.002105,0.059726,0.251360,4.073792,1.744800,6.1980243772268295,64
7,80.232245,0.00106,0.001046,0.002105,0.061810,0.262048,8.173728,1.622336,10.174023918807507,128
8,24.536687,0.00106,0.001046,0.002105,0.062930,0.246944,16.356960,1.403744,18.109570257365704,256
9,58.959151,0.00106,0.001046,0.002105,0.065359,0.258656,33.722561,1.604832,35.688198171555996,512


In [8]:
!nproc
!lscpu
!nvidia-smi

128
Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 57 bits virtual
  Byte Order:             Little Endian
CPU(s):                   128
  On-line CPU(s) list:    0-127
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) Gold 6448Y
    CPU family:           6
    Model:                143
    Thread(s) per core:   2
    Core(s) per socket:   32
    Socket(s):            2
    Stepping:             8
    CPU max MHz:          4100.0000
    CPU min MHz:          800.0000
    BogoMIPS:             4200.00
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 s
                          s ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc 
                          art arch_perfmon pebs bts rep_good nopl xtopology nons
                          top_tsc cpuid aperfmperf tsc_known_freq p